In [101]:
vocaburary_size = 8000
unknown_token = 'UNKNOWN_TOKEN'
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"

In [167]:
import csv
import itertools
import nltk
import numpy as np
import operator
import datetime
import sys

In [103]:
# Read the csv file(open in text mode in python3) and append SENTENCE_START and SENTENCE_END tokens
with open('reddit-comments-2015-08.csv', 'r') as f:
    reader = csv.reader(f, skipinitialspace=True)
    header = next(reader)
    sentences = itertools.chain(*[nltk.sent_tokenize(x[0].lower()) for x in reader])
    sentences = ["%s %s %s" % (sentence_start_token, x, sentence_end_token) for x in sentences]
print('Parsed %d sentences.' % (len(sentences)) )

Parsed 79170 sentences.


In [130]:
print(sentences[0])

SENTENCE_START i joined a new league this year and they have different scoring rules than i'm used to. SENTENCE_END


In [105]:
# Tokenize the sentences into words
tokenized_sentences = [nltk.word_tokenize(sentence) for sentence in sentences]
print('Tokenized into %d words' % (np.sum([len(ts) for ts in tokenized_sentences])))

Tokenized into 1716189 words


In [131]:
print(tokenized_sentences[0])

['SENTENCE_START', 'i', 'joined', 'a', 'new', 'league', 'this', 'year', 'and', 'they', 'have', 'different', 'scoring', 'rules', 'than', 'i', "'m", 'used', 'to', '.', 'SENTENCE_END']


In [107]:
# Count the word frequencies
word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))
print('Found  %d unique word tokens.' % len(word_freq))

Found  65752 unique word tokens.


In [108]:
print(word_freq.most_common(10))

[('SENTENCE_START', 79170), ('SENTENCE_END', 79170), ('.', 67455), ('the', 52338), (',', 52137), ('to', 35568), ('i', 32094), ('a', 31739), ('and', 30007), ('of', 23226)]


In [109]:
# Get the most common words and build index_to_word and word_to_index vectors
vocab = word_freq.most_common(vocaburary_size - 1) 
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([(w, i) for (i, w) in enumerate(index_to_word)])
print('Using vocaburary size: ', vocaburary_size)
print('The least frequent word in our vocaburary is', vocab[-1][0], 'and appeared', vocab[-1][-1], 'times.')

print('The 123th word is', index_to_word[123])
print('The index of', index_to_word[123], 'is', word_to_index[index_to_word[123]])

Using vocaburary size:  8000
The least frequent word in our vocaburary is flagged and appeared 10 times.
The 123th word is most
The index of most is 123


In [110]:
# Replace all words not in our vocaburary with the unknown token
for i, sent in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in word_to_index else unknown_token for w in sent]

In [144]:
# Create training set
X_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in tokenized_sentences])
y_train = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in tokenized_sentences])
print('Example X_train:', ' '.join([index_to_word[i] for i in X_train[17]]), X_train[17])
print('Example y_train:', ' '.join([index_to_word[i] for i in y_train[17]]), y_train[17])

Example X_train: SENTENCE_START what are n't you understanding about this ? ! [0, 51, 27, 16, 10, 853, 53, 25, 34, 69]
Example y_train: what are n't you understanding about this ? ! SENTENCE_END [51, 27, 16, 10, 853, 53, 25, 34, 69, 1]


In [112]:
# RNN paramters
# x_t = [8000,]   equals the vocaburary size
# o_t = [8000,]  same size as the input
# s_t = [100,]     hidden state equals the number of time steps
# U = [100, 8000]
# V = [8000,100]
# W = [100, 100]
#
# s_t = tanh(U * x_t + W * s_t-1)  
# o_t = softmax(V * s_t)
#
# H(hidden size) = 100, C(vocaburary size) = 8000
# total number of parametrs : U + V + W = 100 * 8000 + 8000 * 100 + 100 * 100 = 2HC + H^2

In [137]:
def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a - c)  # To prevent overflow
    sum_exp_a = np.sum(exp_a)
    return exp_a / sum_exp_a

In [170]:
class RNNnumpy:
    def __init__(self, word_dim, hidden_dim = 100, bptt_truncate = 4):
        self.word_dim = word_dim
        self.hidden_dim = hidden_dim
        self.bptt_truncate = bptt_truncate
        # Randomly initialize the network paramters
        self.U = np.random.uniform(-np.sqrt(1./word_dim), np.sqrt(1./word_dim), (hidden_dim, word_dim))
        self.V = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (word_dim, hidden_dim))
        self.W = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (hidden_dim, hidden_dim))
        
    def forward_prop(self, x):
        # The total number of  time steps
        T = len(x)
        # During forward propagation we save all hidden states in s because need them later.
        # we add one additional element for the initial hidden state, which we set to 0.
        s = np.zeros((T + 1, self.hidden_dim))
        s[-1] = np.zeros(self.hidden_dim)
        # The outputs at each time step
        o = np.zeros((T, self.word_dim))
        # for each time step
        for t in np.arange(T):
            # x_t is one hot vector 
            s[t] = np.tanh(self.U[:, x[t]] + self.W.dot(s[t-1]))
            o[t] = softmax(self.V.dot(s[t]))
        return [o, s]
    
    def predict(self, x):
        o, s = self.forward_prop(x)
        return np.argmax(o, axis=1)  # find the index of word wth max probabity (from 8000 vocaburary)
    
    def calculate_total_loss(self, x, y):
        L = 0
        # for each sentence
        for i in np.arange(len(y)):
            o, s = self.forward_prop(x[i])                           
            len_sentence = len(y[i])
            correct_word_index = y[i]
            # we only care about our predictions of the correct words
            correct_word_predictions = o[np.arange(len_sentence), correct_word_index]
            # Add to the loss 
            L += -1 * np.sum(np.log(correct_word_predictions))
        return L
    
    def calculate_loss(self, x, y):
        # Number of words in our text
        N = np.sum((len(y_i) for y_i in y))
        L = self.calculate_total_loss(x, y)
        return L / N 
    
    def bptt(self, x, y):
        # the number of time steps
        T = len(y)
        # forward propagation
        o, s = self.forward_prop(x)
        # derivative (initialized by zeros)
        dLdU = np.zeros(self.U.shape)  # (100, 8000)
        dLdW = np.zeros(self.W.shape)  # (100, 100)
        dLdV = np.zeros(self.V.shape)  # (8000, 100)
        # output
        delta_o = o 
        delta_o[np.arange(len(y)), y] -= 1.
        # for each output backwards
        for t in np.arange(T)[::-1]:
            dLdV += np.outer(delta_o[t], s[t].T)
            # initial delta calculation dL/dz
            delta_t = self.V.T.dot(delta_o[t]) * (1 - (s[t] ** 2))
            # bptt
            for bptt_step in np.arange(max(0, t - self.bptt_truncate), t+1)[::-1]:
                #print("Bptt step t = %d, bptt step = %d " % (t, bptt_step))
                dLdW += np.outer(delta_t, s[bptt_step - 1])
                dLdU[:, x[bptt_step]] += delta_t
                # update delta_t for next step dL/dz at t -1 
                delta_t = self.W.T.dot(delta_t) * (1 - s[bptt_step - 1] ** 2)
        return [dLdU, dLdV, dLdW]
    
    def gradient_check(self, x, y, h=0.001, error_th=0.01):
        # calc the grads using backprop.
        bptt_grads = self.bptt(x, y)
        # list of all params we want to check
        model_params = ['U', 'V', 'W']
        # gradient check for each params
        for pidx, pname in enumerate(model_params):
            # get the actual parameter value from the mode
            parameter = operator.attrgetter(pname)(self)
            print('gradient check for %s with size %d.' % (pname, np.prod(parameter.shape)))
            # iterate over each element of the parameter matrix
            it = np.nditer(parameter, flags=['multi_index'], op_flags=['readwrite'])
            while not it.finished:
                ix = it.multi_index
                # save the original value
                original_value = parameter[ix]
                # estimate the gradient using (f(x+h) - f(x-h)) / (2*h)
                parameter[ix] = original_value + h
                gradplus = self.calculate_total_loss([x], [y])
                parameter[ix] = original_value - h
                gradminus = self.calculate_total_loss([x], [y])
                estimated_grad = (gradplus - gradminus) / (2 * h)
                # reset parameter to original value
                parameter[ix] = original_value
                # the gradient for this parameter calculated using backprop
                backprop_grad = bptt_grads[pidx][ix]
                # calculate the relative error (|x -y| / (|x| + |y|))
                relative_error = np.abs(backprop_grad - estimated_grad) / (np.abs(backprop_grad) + np.abs(estimated_grad))
                # if the error is too large fail the gradient check
                if relative_error > error_th:
                    print("gradient check error: parameter = %s, ix = %s " %(pname, ix))
                    print("+h loss =", gradplus)
                    print("-h loss=", gradminus)
                    print("estimated grad =", estimated_grad)
                    print("backprop grad=", backprop_grad)
                    print("relative error=", relative_error)
                    return
                it.iternext()
            print('gradient check for parameter %s passed.' % (pname))

    # Performs one step of SGD
    def sgd_step(self, x, y, learning_rate):
        # calculate the grads
        dLdU, dLdV, dLdW = self.bptt(x, y)
        # change parameters according to grads and learning rate
        self.U -= learning_rate * dLdU
        self.V -= learning_rate * dLdV
        self.W -= learning_rate * dLdW

In [171]:
print(X_train.shape)
print(len(X_train[10]), X_train[10])
print(len(X_train[123]), X_train[123])

(79170,)
45 [0, 72, 63, 13, 124, 5, 26, 1121, 208, 5, 324, 3, 329, 4, 112, 32, 75, 7, 4743, 4, 8, 84, 52, 9, 7, 3151, 1014, 491, 7535, 8, 133, 49, 3092, 4, 10, 95, 51, 4, 128, 17, 37, 314, 575, 2, 40]
11 [0, 56, 29, 16, 38, 412, 70, 211, 1151, 190, 2]


In [172]:
# Try an implementation of RNNnumpy class
np.random.seed(10)
model = RNNnumpy(vocaburary_size)
o, s = model.forward_prop(X_train[10])
print(o.shape, s.shape)
preds = model.predict(X_train[10])
print(preds)

# calculate the loss
print('Expected loss for random predictions:', np.log(vocaburary_size))
print('Actual loss:', model.calculate_loss(X_train[:1000], y_train[:1000]))

(45, 8000) (46, 100)
[1284 5221 7653 7430 1013 3562 7366 6198 5028 3376 7299 6722 6892 3198 5738
 5853 2926  261 2538 1653 3177 5376 3522  477 7051 1830 7609 6607 1201 4221
 1900 6176 3417 3256 4864 2182 6569 2800 2752 6821 4437 7021 2864 7071 2525]
Expected loss for random predictions: 8.98719682066
Actual loss: 8.98742855675


In [173]:
grad_check_vocab_size = 100
np.random.seed(10)
model = RNNnumpy(grad_check_vocab_size, 10, bptt_truncate=1000)
model.gradient_check([0,1,2,3], [1,2,3,4])

gradient check for U with size 1000.


/Users/satojkovic/.pyenv/versions/3.6.1/lib/python3.6/site-packages/ipykernel/__main__.py:103: RuntimeWarning: invalid value encountered in double_scalars


gradient check for parameter U passed.
gradient check for V with size 1000.
gradient check for parameter V passed.
gradient check for W with size 100.
gradient check for parameter W passed.


In [174]:
np.random.seed(10)
model = RNNnumpy(vocaburary_size)
%timeit model.sgd_step(X_train[10], y_train[10], 0.005)

1 loop, best of 3: 248 ms per loop


In [175]:
# - model: The RNN model instance
# - x_train: The training dataset
# - y_train: The training data labels
# - learning_rate: The initial learning rate for SGD
# - nepoch: Number of times to iterate through over the complete dataset
# - evaluate_loss_after: Evaluate the loss after this many epochs
def train_with_sgd(model, x_train, y_train, learning_rate=0.005, nepoch=100, evaluate_loss_after=5):
    losses = []
    num_examples_seen = 0
    for epoch in range(nepoch):
        if epoch % evaluate_loss_after == 0:
            loss = model.calculate_loss(x_train, y_train)
            losses.append((num_examples_seen, loss))
            time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            print('%s: loss after num_examples_seen=%d epoch=%d: %f' % (time, num_examples_seen, epoch, loss))
            # adjust the learning rate if loss increases
            if len(losses) > 1 and losses[-1][1] > losses[-2][1]:
                learning_rate = learning_rate * 0.5
                print('setting learning rate to %f' % (learning_rate))
            sys.stdout.flush()
        # for each training example
        for i in range(len(y_train)):
            # one SGD step
            model.sgd_step(x_train[i], y_train[i], learning_rate)
            num_examples_seen += 1

In [176]:
np.random.seed(10)
model = RNNnumpy(vocaburary_size)
losses = train_with_sgd(model, X_train[:100], y_train[:100], nepoch=10, evaluate_loss_after=1)

2017-03-28 22:35:17: loss after num_examples_seen=0 epoch=0: 8.987418
2017-03-28 22:35:30: loss after num_examples_seen=100 epoch=1: 8.976055
2017-03-28 22:35:42: loss after num_examples_seen=200 epoch=2: 8.959715
2017-03-28 22:35:57: loss after num_examples_seen=300 epoch=3: 8.929265
2017-03-28 22:36:11: loss after num_examples_seen=400 epoch=4: 8.840789
2017-03-28 22:36:24: loss after num_examples_seen=500 epoch=5: 6.748881
2017-03-28 22:36:36: loss after num_examples_seen=600 epoch=6: 6.250084
2017-03-28 22:36:49: loss after num_examples_seen=700 epoch=7: 5.992692
2017-03-28 22:37:05: loss after num_examples_seen=800 epoch=8: 5.822810
2017-03-28 22:37:20: loss after num_examples_seen=900 epoch=9: 5.704647
